## Import package

In [1]:
import pandas as pd
import os
import numpy as np
from collections import defaultdict
from tqdm import tqdm
import gc

pd.options.mode.chained_assignment = None

In [2]:
source_path = '../data/mimiciii/source_data/'
dest_path = '../data/mimiciii/preprocess_data/'

## Split CHARTEVENTS to several dataset

In [3]:
count = 1
for reader in tqdm(pd.read_csv(source_path + 'CHARTEVENTS.csv', chunksize = 40000000)):
    reader.to_csv(dest_path + 'CHARTEVENTS' + str(count) + '.csv', index=False)
    count += 1

0it [00:00, ?it/s]/home/toooot/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (8,13,14) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
4it [07:01, 105.77s/it]/home/toooot/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (13) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
6it [10:41, 108.19s/it]/home/toooot/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (8,10,14) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
9it [14:45, 98.36s/it] 


## Load Data

In [4]:
paths = sorted(os.listdir(source_path))
remove_paths = []
for path in paths:
    if path[-4:] != '.csv':
        remove_paths.append(path)
for path in remove_paths:
    paths.remove(path)
paths.remove('CALLOUT.csv')
paths.remove('CAREGIVERS.csv')
paths.remove('CHARTEVENTS.csv')
paths.remove('CPTEVENTS.csv')
paths.remove('DATETIMEEVENTS.csv')
paths.remove('SERVICES.csv',)
paths.remove('TRANSFERS.csv')
paths.remove('D_CPT.csv')

In [5]:
datas = []
for path in tqdm(paths):
    datas.append(pd.read_csv(source_path + path))
    
for path in tqdm(sorted(os.listdir(dest_path))):
    datas.append(pd.read_csv(dest_path + path))

paths += sorted(list(os.listdir(dest_path)))

 44%|██████████████████████████████████████████████████████████████████████████▏                                                                                            | 8/18 [00:15<00:00, 28.00it/s]/home/toooot/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (7,9,17,20,21) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
 67%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████▋                                                       | 12/18 [00:38<00:27,  4.55s/it]/home/toooot/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
 78%|██████████████████████████████████████████████████████████████████████████████████████████████

## Clean Data to remain SEPSIS patient and sort Data

In [6]:
i = paths.index('D_ICD_DIAGNOSES.csv')
D_ICD_DIAGNOSES = datas[i].copy()
D_ICD_DIAGNOSES.drop(D_ICD_DIAGNOSES[['sepsis' not in s.lower() for s in D_ICD_DIAGNOSES['LONG_TITLE']]].index, inplace=True)
D_ICD_DIAGNOSES.to_csv(dest_path + paths[i], index=False)

In [7]:
i = paths.index('DIAGNOSES_ICD.csv')
datas[i].sort_values(by=['SUBJECT_ID', 'SEQ_NUM'], inplace=True)
DIAGNOSES_ICD = datas[i].copy()

patient = set()
for j in tqdm(DIAGNOSES_ICD.index):
    for code in D_ICD_DIAGNOSES['ICD9_CODE']:
        if str(DIAGNOSES_ICD.loc[j]['ICD9_CODE']) == code:
            patient.add((DIAGNOSES_ICD.loc[j]['SUBJECT_ID'], DIAGNOSES_ICD.loc[j]['HADM_ID']))
            break

drop_index = []
for j in tqdm(DIAGNOSES_ICD.index):
    subject_id = DIAGNOSES_ICD.loc[j, 'SUBJECT_ID']
    hadm_id = DIAGNOSES_ICD.loc[j, 'HADM_ID']
    if (subject_id, hadm_id) not in patient:
        drop_index.append(j)
DIAGNOSES_ICD.drop(drop_index, inplace=True)
DIAGNOSES_ICD.to_csv(dest_path + paths[i], index=False)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 651047/651047 [00:04<00:00, 156859.57it/s]


In [8]:
subject_id, hadm_id = [], []
for sub, hadm in patient:
    subject_id.append(sub)
    hadm_id.append(hadm)
dataset = pd.DataFrame({'SUBJECT_ID' : subject_id, 'HADM_ID' : hadm_id})
dataset.sort_values(by=['SUBJECT_ID', 'HADM_ID'], inplace=True)
dataset.index = range(len(dataset))
dataset.to_csv('../data/mimiciii/patient.csv', index=False)

In [9]:
dict1 = defaultdict(list)
for i in range(len(dataset)):
    dict1[dataset['SUBJECT_ID'][i]].append(dataset['HADM_ID'][i])

for i in tqdm(range(len(datas))):
    if i == paths.index('DIAGNOSES_ICD.csv'):
        continue
    if 'SUBJECT_ID' in datas[i].columns and 'HADM_ID' in datas[i].columns:
        drop_index = []
        for j in datas[i].index:
            if datas[i]['HADM_ID'][j] not in dict1[datas[i]['SUBJECT_ID'][j]]:
                drop_index.append(j)
        datas[i].drop(drop_index, inplace=True)
        del drop_index
        # datas[i].sort_values(by=['SUBJECT_ID', 'HADM_ID'], inplace=True)
    elif 'SUBJECT_ID' in datas[i].columns:
        datas[i] = datas[i][[id in dataset['SUBJECT_ID'].values for id in datas[i]['SUBJECT_ID']]]
        # datas[i].sort_values(by=['SUBJECT_ID'], inplace=True)

    datas[i].to_csv(dest_path + paths[i], index=False)
    
del dict1
gc.collect()
        
i = paths.index('D_ICD_PROCEDURES.csv')
datas[i].sort_values(by=['ICD9_CODE'], inplace=True)
datas[i].to_csv(dest_path + paths[i], index=False)

i = paths.index('D_ITEMS.csv')
datas[i].sort_values(by=['ITEMID'], inplace=True)
datas[i].to_csv(dest_path + paths[i], index=False)

i = paths.index('D_LABITEMS.csv')
datas[i].sort_values(by=['ITEMID'], inplace=True)
datas[i].to_csv(dest_path + paths[i], index=False)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 27/27 [32:30<00:00, 72.25s/it]
